<table align="left"><td><a target="_blank" href="https://colab.research.google.com/github/superannotateai/model-deployment-tutorials/blob/main/JETSON/SuperAnnotate_JETSON_MobileNetSSD_Deployment.ipynb"><img src="https://user-images.githubusercontent.com/25985824/104791629-6e618700-5769-11eb-857f-d176b37d2496.png" height="32" width="32"> Try in Google Colab</a></td></table>

# 0. Preparation Steps
Before starting the pipeline you need to download tgz archives of TensorRT and CUDNN and place them in your google drive. (This notebook is tested with TensorRT 7.2.3.4 and CUDNN-11.2 versions)

# 1. Install Prerequisites 
(Please click on **RESTART RUNTIME** button when it appears in the output of this code block)



In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/brokenerk/TRT-SSD-MobileNetV2.git
!cp /content/drive/MyDrive/TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar.gz /content/
!cp /content/drive/MyDrive/cudnn-11.2-linux-x64-v8.1.1.33.tgz /content/

!tar -xzvf cudnn-11.2-linux-x64-v8.1.1.33.tgz

!sudo cp cuda/include/cudnn*.h /usr/local/cuda/include 
!sudo cp -P cuda/lib64/libcudnn* /usr/local/cuda/lib64 
!sudo chmod a+r /usr/local/cuda/include/cudnn*.h /usr/local/cuda/lib64/libcudnn*

!tar xzvf TensorRT-7.2.3.4.Ubuntu-18.04.x86_64-gnu.cuda-11.0.cudnn8.1.tar.gz
os.environ['LD_LIBRARY_PATH'] += ':/content/TensorRT-7.2.3.4/lib/'

%cd /content/TensorRT-7.2.3.4/python/
!sudo pip3 install tensorrt-7.2.3.4-cp37-none-linux_x86_64.whl

%cd /content/TensorRT-7.2.3.4/uff/
!sudo pip3 install uff-0.6.9-py2.py3-none-any.whl

%cd /content/TensorRT-7.2.3.4/graphsurgeon/
!sudo pip3 install graphsurgeon-0.4.5-py2.py3-none-any.whl
%cd /content/TensorRT-7.2.3.4/onnx_graphsurgeon
!sudo pip3 install onnx_graphsurgeon-0.2.6-py2.py3-none-any.whl

!pip install superannotate
!pip install google-resumable-media==0.5.0

!pip install numpy==1.17.5
!pip install tf_slim
!sudo apt-get update
!sudo apt-get install libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip libjpeg8-dev liblapack-dev libblas-dev gfortran
!sudo apt-get install python3-pip
!pip3 install -U pip testresources setuptools==49.6.0
!pip3 install future==0.18.2 mock==3.0.5 h5py==2.10.0 keras_preprocessing==1.1.1 keras_applications==1.0.8 gast==0.2.2 futures protobuf pybind11
!pip3 install --pre --extra-index-url https://developer.download.nvidia.com/compute/redist/jp/v43 'tensorflow<2'
!pip3 install pycuda

# 2. Setup your SuperAnnotate token, project names of trianing images and desired output model name

In [ ]:
%tensorflow_version 1.x

TOKEN = "For Pro Users, the Token-ID can be generated in the teams section"
PROJECT_NAMES = ["Mask - batch 1", "Mask - batch 2"]
OUTPUT_MODEL = "sa-tutorial"

# 3. Download and preprocess the data for training

In [ ]:
import superannotate as sa
import json
import os 
from shutil import copy, make_archive, rmtree
import re
import glob
import urllib.request
import tarfile
from pathlib import Path
import tensorflow as tf
import random
import re

os.environ['CUDA_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['CUDNN_INSTALL_DIR']= "/usr/local/cuda/"
os.environ['LD_LIBRARY_PATH']+= ":/usr/lib64-nvidia/:/content/TensorRT-7.2.3.4/include/:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/"
os.environ['PATH']+= ":/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib:/content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/"
!sudo rm /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7
!sudo ln -s /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7.2.3 /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/libnvinfer.so.7
!sudo cp -r /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/lib/*.* /usr/lib/ 
!sudo cp -r /content/TensorRT-7.2.3.4/targets/x86_64-linux-gnu/bin/*.* /usr/bin/ 

proj_folder = dict(zip(PROJECT_NAMES, ["proj_" + str(i) for i in range(len(PROJECT_NAMES))]))
token_json = {"token": TOKEN}
with open('sa_config.json', 'w') as f:
  json.dump(token_json, f)

sa.init('sa_config.json')

for project_name, folder_name in proj_folder.items():
  if not os.path.exists('./training_data/' + folder_name):
    os.makedirs('training_data/' + folder_name)
  completed_images = sa.search_images(project_name, annotation_status="Completed")
  for completed_image in completed_images:
    sa.download_image(project_name, completed_image, local_dir_path='./training_data/' + folder_name)
  export = sa.prepare_export(project_name, annotation_statuses=["Completed"])
  sa.download_export(project_name, export, './training_data/' + folder_name)

%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'

%cd /content/models/research/
from object_detection.utils import dataset_util, label_map_util
data_base = Path("/content/training_data")
class_names = set()
column_names = ["filename", "width", "height", "class", "xmin", "ymin", "xmax", "ymax"]
for project_dir in data_base.glob("*"):
  with open(project_dir / "classes" / "classes.json") as f:
    proj_class_data = json.load(f)
  for class_entry in proj_class_data:
    class_names.add(class_entry["name"])
class_names = sorted(list(class_names))
pbtxt_content = ""
for i, class_name in enumerate(class_names):
  pbtxt_content = (pbtxt_content + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name))
pbtxt_content = pbtxt_content.strip()
with (data_base / "label_map.pbtxt").open(mode="w") as f:
  f.write(pbtxt_content)

tf_record_dir = data_base / "annotations"
if not tf_record_dir.exists():
  tf_record_dir.mkdir()

writers = [tf.python_io.TFRecordWriter(str(tf_record_dir / "train.record")), tf.python_io.TFRecordWriter(str(tf_record_dir / "test.record"))]
label_map = label_map_util.load_labelmap(str(data_base / "label_map.pbtxt"))
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=90, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map = {}
for k, v in category_index.items():
    label_map[v.get("name")] = v.get("id")
    
for project_dir in list(data_base.glob("*")):
  annot_paths = list(project_dir.glob("*___objects.json"))
  for annot_path in annot_paths:
    with open(annot_path) as f:
      annot_data = json.load(f)
    file_name = annot_path.name[:-15]
    with tf.gfile.GFile(str(project_dir / file_name), "rb") as fid:
      encoded_jpg = fid.read()
    width = annot_data["metadata"]["width"]
    height = annot_data["metadata"]["height"]
    image_format = Path(file_name).suffix[1:].encode("utf8")
    file_name = file_name.encode("utf8")
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    for inst_data in annot_data["instances"]:
      points = inst_data["points"]
      xmin, xmax = min(points["x1"], points["x2"]), max(points["x1"], points["x2"])
      ymin, ymax = min(points["y1"], points["y2"]), max(points["y1"], points["y2"])
      xmins.append(xmin / width)
      xmaxs.append(xmax / width)
      ymins.append(ymin / height)
      ymaxs.append(ymax / height)
      classes_text.append(inst_data["className"].encode("utf8"))
      class_index = label_map.get(inst_data["className"])
      classes.append(class_index)
    tf_example = tf.train.Example(
      features=tf.train.Features(
        feature={
          "image/height": dataset_util.int64_feature(height),
          "image/width": dataset_util.int64_feature(width),
          "image/filename": dataset_util.bytes_feature(file_name),
          "image/source_id": dataset_util.bytes_feature(file_name),
          "image/encoded": dataset_util.bytes_feature(encoded_jpg),
          "image/format": dataset_util.bytes_feature(image_format),
          "image/object/bbox/xmin": dataset_util.float_list_feature(xmins),
          "image/object/bbox/xmax": dataset_util.float_list_feature(xmaxs),
          "image/object/bbox/ymin": dataset_util.float_list_feature(ymins),
          "image/object/bbox/ymax": dataset_util.float_list_feature(ymaxs),
          "image/object/class/text": dataset_util.bytes_list_feature(classes_text),
          "image/object/class/label": dataset_util.int64_list_feature(classes),
        }
      )
    )
    writer_id = random.choices([0, 1], weights=[0.9, 0.1])[0]
    writers[writer_id].write(tf_example.SerializeToString())

for writer in writers:
  writer.close()

test_record_fname = '/content/training_data/annotations/test.record'
train_record_fname = '/content/training_data/annotations/train.record'
label_map_pbtxt_fname = '/content/training_data/label_map.pbtxt'
num_steps = 5000
num_eval_steps = 50
batch_size = 24
MODEL = 'ssd_mobilenet_v2_coco_2018_03_29'
pipeline_file = 'ssd_mobilenet_v2_coco.config'

%cd /content/models/research

MODEL_FILE = Path(MODEL + '.tar.gz')
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = Path('/content/models/research/pretrained_model')

if not MODEL_FILE.exists():
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE.name, MODEL_FILE.name)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

MODEL_FILE.unlink()
if DEST_DIR.exists():
  rmtree(DEST_DIR)
Path(MODEL).replace(DEST_DIR)

fine_tune_checkpoint = DEST_DIR / "model.ckpt"

pipeline_fname = Path("/content/models/research/object_detection/samples/configs/") / pipeline_file
iou_threshold = 0.5
num_classes = len(category_index)
with pipeline_fname.open() as f:
  s = f.read()
with pipeline_fname.open(mode="w") as f:
  # fine_tune_checkpoint
  s = re.sub('fine_tune_checkpoint: ".*?"',
              'fine_tune_checkpoint: "{}"'.format(str(fine_tune_checkpoint)), s)

  # tfrecord files train and test.
  s = re.sub(
      '(input_path: ".*?)(train.record)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
  s = re.sub(
      '(input_path: ".*?)(val.record)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

  # label_map_path
  s = re.sub(
      'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

  # Set training batch_size.
  s = re.sub('batch_size: [0-9]+',
              'batch_size: {}'.format(batch_size), s)

  # Set training steps, num_steps
  s = re.sub('num_steps: [0-9]+',
              'num_steps: {}'.format(num_steps), s)

  # Set number of classes num_classes.
  s = re.sub('num_classes: [0-9]+',
              'num_classes: {}'.format(num_classes), s)
  # Set number of classes num_classes.
  s = re.sub('iou_threshold: [0-9].[0-9]+',
              'iou_threshold: {}'.format(iou_threshold), s)

  f.write(s)

# 4. Train the model

In [ ]:
model_dir = 'training/'
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

# 5. Generate and download the files needed for the OAK-D device

In [ ]:
import numpy as np
import requests
from google.colab import files

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)

!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")

In [ ]:
%cd /content/TRT-SSD-MobileNetV2
import uff
import tensorrt as trt
import graphsurgeon as gs
import config as model
from google.colab import files

# initialize
TRT_LOGGER = trt.Logger(trt.Logger.INFO)
trt.init_libnvinfer_plugins(TRT_LOGGER, '')
runtime = trt.Runtime(TRT_LOGGER)


# compile model into TensorRT
if not os.path.isfile(OUTPUT_MODEL + '.bin'):
    dynamic_graph = model.add_plugin(gs.DynamicGraph(pb_fname))
    uff_model = uff.from_tensorflow(dynamic_graph.as_graph_def(), model.output_name, output_filename='tmp.uff')

    with trt.Builder(TRT_LOGGER) as builder, builder.create_network() as network, trt.UffParser() as parser:
        builder.max_workspace_size = 1 << 28
        builder.max_batch_size = 1
        builder.fp16_mode = True

        parser.register_input('Input', model.dims)
        parser.register_output('MarkOutput_0')
        parser.parse('tmp.uff', network)
        engine = builder.build_cuda_engine(network)

        buf = engine.serialize()
        with open(OUTPUT_MODEL + '.bin') as f:
            f.write(buf)

files.download(OUTPUT_MODEL + '.bin')

# 6. Test the custom model

Afterwards you can load the binary file with tensorrt on jetson devices and integrate it to your pipelines.
